In [ ]:
pip install pandas==1.3.4

In [33]:
%load_ext autoreload
%autoreload 2
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'


In [35]:
# !pip install fcsparser
# !pip install pandas
# !pip install scanpy
# !pip install torch
!pip install pycombat

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import scanpy as sc
import os
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from torch.autograd import Variable

In [3]:
# IF this doesn't work, try reading fcs natively using python
data_dir = f'/home/razgaon/ondemand/data/sys/myjobs/projects/default/3/'
NSCLC_pickle = f'{data_dir}NSCLC_prepared_data_test.pickle'
PBMC_pickle = f'{data_dir}PBMC_prepared_data_test.pickle'
NSCLC = pd.read_pickle(NSCLC_pickle)
PBMC = pd.read_pickle(PBMC_pickle)


# Try to add an id that maps to the fcs file name


In [4]:
PBMC_df = PBMC.sample(100000)
NSCLC_df = NSCLC.sample(100000)

In [ ]:
#check if pycombat requires normalization first for each dataset, or after
#concat (keep all markers), batch normalize on X with NaNs, umap to confirm quality,  min-max scale, then vae, umap to confirm quality

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
excluded_markers = ['id', 'Time', 'Eventlength', 'sample', 'batch']
NSCLC_markers = [col for col in NSCLC_df.columns if col not in excluded_markers]
NSCLC_df[NSCLC_markers] = scaler.fit_transform(NSCLC_df[NSCLC_markers])

PBMC_markers = [col for col in PBMC_df.columns if col not in excluded_markers]
PBMC_df[PBMC_markers] = scaler.fit_transform(PBMC_df[PBMC_markers])


In [24]:
df = pd.concat([PBMC_df, NSCLC_df])

In [7]:

class MyDataset(Dataset):
 
    def __init__(self,df):
        x=df.values
        self.x_train=torch.tensor(x,dtype=torch.float32)

    def __len__(self):
        return len(self.x_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx]


In [25]:
# Convert metals to their marker names to ensure consistency when merging dataframes
overlap = list(set(PBMC_markers).intersection(set(NSCLC_markers)))

In [27]:
# markers = NSCLC_markers + PBMC_markers
dataset = MyDataset(df[overlap])
# # Data loader
train_loader = DataLoader(dataset, batch_size=1000, shuffle=True)
test_loader = DataLoader(dataset, batch_size=1000, shuffle=True)


,HLADR,CTLA4,CD25,CXCR5,CXCR3,CD127,TIM3,CD45RA,ICOS,CD4,CD3,CCR6,PD1
340740,0.103427,0.000000,0.174617,0.141331,0.038327,0.028797,0.476414,0.173947,0.000000,0.120253,0.241595,0.000000,0.000000
790657,0.000000,0.072121,0.214664,0.000000,0.000000,0.000000,0.000000,0.632033,0.000000,0.550223,0.615384,0.113164,0.000000
890756,0.119222,0.105180,0.000000,0.038327,0.219516,0.056529,0.460911,0.686022,0.000000,0.077609,0.644587,0.113164,0.083909
465676,0.299882,0.187862,0.097765,0.917641,0.000000,0.028797,0.000000,0.769668,0.048604,0.027109,0.031381,0.745582,0.000000
425217,0.220598,0.036739,0.000000,0.000000,0.038327,0.000000,0.316560,0.000000,0.095411,0.229726,0.616662,0.000000,0.000000


In [28]:
class VAE(nn.Module):
    def __init__(self, x_dim, latent_dim=5, hdim=50):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.latent_dim = latent_dim
        self.fc1 = nn.Linear(x_dim , hdim)
        self.fc21 = nn.Linear(hdim, latent_dim)
        self.fc22 = nn.Linear(hdim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hdim)
        self.fc4 = nn.Linear(hdim, x_dim)
        
    def encode(self, x):
        x_ = torch.cat([x], dim=-1)
        h1 = F.relu(self.fc1(x_))
        return self.fc21(h1), self.fc22(h1)
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.fc4(h3)
    def reparameterize(self, mu, var):
#         if self.distance == 'standard':
        if self.training:
            std = var.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            result = eps.mul(std).add_(mu)
            return result
        else:
            return mu
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.x_dim))
        latent = self.reparameterize(mu, logvar)
        reconstructed = self.decode(latent)
        return reconstructed, latent

In [29]:
def compute_kernel(x, y, device):
    x_size = x.to(device).size(0)
    y_size = y.to(device).size(0)
    dim = x.size(1)
    x = x.unsqueeze(1)
    y = y.unsqueeze(0)
    tiled_x = x.expand(x_size, y_size, dim).to(device)
    tiled_y = y.expand(x_size, y_size, dim).to(device)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input)

def compute_mmd(x, y, device):
    x_kernel = compute_kernel(x, x, device)
    y_kernel = compute_kernel(y, y, device)
    xy_kernel = compute_kernel(x, y, device)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

def vae_loss_function(inputs, encoded, decoded, true_samples, device):
    #reconstruction loss - decoding
    nll = (decoded - inputs).pow(2).mean()
    #distance + reconstruction loss
    mmd = compute_mmd(true_samples, encoded.to(device), device)
    return(nll, mmd)

In [30]:
def train(net, learning_rate, epochs, train_loader, test_loader, optimizer = 'Adam'):
    if optimizer == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr = learning_rate)
    elif optimizer == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr = learning_rate, 
                              momentum = .95, nesterov = True)
    
    for epoch in range(epochs):
        training_loss = 0
        training_reconstruction_error = 0
        training_mmd = 0

        net.train()
        for batchnum, X in enumerate(train_loader):
            optimizer.zero_grad()
            reconstruction, mu = net(X)
            true_samples = torch.randn(mu.shape)
            reconstruction_error, mmd = vae_loss_function(inputs = X, encoded=mu, decoded=reconstruction, true_samples=true_samples , device=device)
            loss = reconstruction_error + mmd
            loss.backward()
            
            optimizer.step()

            training_reconstruction_error += reconstruction_error
            training_mmd  += mmd
            training_loss += loss

        training_reconstruction_error /= (batchnum+1)
        training_mmd  /= (batchnum+1)
        training_loss /= (batchnum+1)
        print('Training loss for epoch %i is %.8f, Reconstruction is %.8f, mmd is %.8f'%(epoch, training_loss, training_reconstruction_error, training_mmd) )
        
        #Testing loop

        testing_reconstruction_error = 0
        testing_mmd = 0

        with torch.no_grad():
            for batchnum, X in enumerate(test_loader):
                reconstruction, mu = net(X)
                true_samples = torch.randn(mu.shape)
                reconstruction_error, mmd = vae_loss_function(inputs = X, encoded=mu, decoded=reconstruction, true_samples=true_samples , device=device)

                testing_reconstruction_error += reconstruction_error
                testing_mmd += mmd
            
            testing_reconstruction_error /= (batchnum+1)
            testing_mmd /= (batchnum+1)
            print('Testing loss for epoch %i is %.8f, Reconstruction is %.8f, mmd is %.8f'%(epoch, testing_reconstruction_error + testing_mmd, testing_reconstruction_error, testing_mmd) )
        

In [31]:
net = VAE(x_dim=dataset.x_train.shape[1])

In [34]:
train(net, learning_rate = .0001, epochs = 30,
      train_loader = train_loader, test_loader = test_loader)

Training loss for epoch 0 is 0.12598665, Reconstruction is 0.12377081, mmd is 0.00221584
Testing loss for epoch 0 is 0.08257637, Reconstruction is 0.08142467, mmd is 0.00115170
Training loss for epoch 1 is 0.06481549, Reconstruction is 0.06378343, mmd is 0.00103205
Testing loss for epoch 1 is 0.05272894, Reconstruction is 0.05177472, mmd is 0.00095422
Training loss for epoch 2 is 0.04706269, Reconstruction is 0.04607858, mmd is 0.00098410
Testing loss for epoch 2 is 0.04296221, Reconstruction is 0.04200570, mmd is 0.00095652
Training loss for epoch 3 is 0.04066073, Reconstruction is 0.03971995, mmd is 0.00094077
Testing loss for epoch 3 is 0.03892866, Reconstruction is 0.03799993, mmd is 0.00092872
Training loss for epoch 4 is 0.03783739, Reconstruction is 0.03695334, mmd is 0.00088404
Testing loss for epoch 4 is 0.03694220, Reconstruction is 0.03608163, mmd is 0.00086057
Training loss for epoch 5 is 0.03630543, Reconstruction is 0.03547177, mmd is 0.00083365
Testing loss for epoch 5 i

In [36]:
torch.save(net, 'model_state')

In [37]:
# Model class must be defined somewhere
model = torch.load('model_state')
model.eval()

VAE(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (fc21): Linear(in_features=50, out_features=5, bias=True)
  (fc22): Linear(in_features=50, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=13, bias=True)
)

In [38]:
dataset.x_train

tensor([[0.1034, 0.0000, 0.1746,  ..., 0.2416, 0.0000, 0.0000],
        [0.0000, 0.0721, 0.2147,  ..., 0.6154, 0.1132, 0.0000],
        [0.1192, 0.1052, 0.0000,  ..., 0.6446, 0.1132, 0.0839],
        ...,
        [0.1249, 0.2198, 0.0829,  ..., 0.6044, 0.1242, 0.2148],
        [0.2660, 0.1001, 0.0000,  ..., 0.1282, 0.0000, 0.2941],
        [0.3818, 0.4265, 0.0225,  ..., 0.6453, 0.0000, 0.3827]])

In [39]:
r, latent = model(dataset.x_train)

In [40]:
torch.save(latent, 'latent.pt')

In [41]:
latent.shape

torch.Size([200000, 5])

In [42]:
torch.save(dataset.x_train, 'dataset_xtrain.pt')